# This file project will take in all of the emails and pull out the links that are relevant. It will shorten the number of articles you will need to research and save up some time. Follow the steps below. 

Input: A file with the email copy and pasted

Output: 
- A file with the relevant links AND
- A file with the links that you will have to manually look through since the Colab notebook was unable to

FIRST: Copy all the emails into a single file(.csv, .fasta, etc). 

SECOND: Add file to google drive using "File Upload". DO NOT put the file in any folders. 

In [ ]:
#@title THIRD: Enter name of file below:

file_name = 'aug03_2022.raw.fasta' #@param {type:"string"}

#Import and mount to drive

#import os, sys
#from sklearn.preprocessing import MinMaxScaler
#from xgboost import XGBClassifier
#from sklearn.model_selection import train_test_split
#from sklearn.metrics import accuracy_score
#from sklearn.metrics import confusion_matrix
#import seaborn as sn
#import matplotlib.pyplot as plt

from google.colab import drive
import pandas as pd
import numpy as np

!pip install newspaper3k
!pip install summa

import newspaper
import nltk
import json
import re

from summa import summarizer, keywords
nltk.download('punkt')

drive.mount('/content/drive')
%cd /content/drive/My Drive

infile = open(file_name)

alllines = []

urls = []
summaries = []
temp = ""
for line in infile:
  if line != '\n':
    alllines.append(line)
    temp += line
alllines = temp.split(' \n')

for line in alllines:
  if "https" in line:
    temppy = line.split('\n')
    urls.append(temppy[1])
    summaries.append(temppy[0])

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive


In [ ]:
#@title FOURTH: Enter topic names you would like to search for--Separate with a comma:

terms = "Russia, Wagner, Putin, Moscow, Ukrain, Zelenskyy, Kyiv" #@param {type:"string"}


def get_sentences(text):
  #print(text)
  paragraphs = []
  paragraphs = text.split("\n\n")

  warTerms = terms.split(", ")
  
  countries = ["African Union","Algeria","Angola","Benin","Botswana","Burkina Faso","Burundi","Verde","Cameroon",
          "Central African Republic","CAR", "Chad","Comoros","Congo","Cote d’Ivoire","Djibouti","Egypt","Equatorial Guinea","Eritrea","Eswatini",
          "Ethiopia","Gabon","Gambia","Ghana","Guinea","Guinea-Bissau","Kenya","Lesotho","Liberia","Libya","Madagascar","Malawi","Mali",
          "Mauritania","Mauritius","Morocco","Mozambique","Namibia","Niger","Nigeria","Rwanda","Sao Tome","Principe","Senegal","Seychelles",
          "Sierra Leone","Somalia","South Africa","South Sudan","Sudan","Tanzania","Togo","Tunisia","Uganda","Western Saharah","Yemen","Zambia",
          "Zimbabwe"]

  textList = []
  warTermPresent = False

 #print(warTerms)

  for para in paragraphs:
    #print(para)
    for x in warTerms:
      if x in para:
        textList.append(para)
        warTermPresent = True
  intext = 0
  reducedList = []
  if warTermPresent == True:
    intext = 1
    for para in textList:
      for c in countries:
        if c in para:
          reducedList.append(para)

  test = list(set(reducedList))
  return intext

In [ ]:
#@title **Some URLS do not work with this code. Here is the list of them: 

dataa = [[]]
manual = []

for line in urls:
  article = newspaper.Article(url=line, language='en')
  sum = summaries.pop(0)
  try:
    article.download()
    article.parse()
    article.nlp()
    temp = []
    #'url', 'title', 'summary from email', 'sample summary', 'mention_of_vars'
    temp.append(line)
    temp.append(article.title)
    temp.append(sum)
    sim = get_sentences(article.text)
    temp.append(article.summary)
    mis = get_sentences(sum)
    temp.append(sim+mis)
    
    dataa.append(temp)

  except:
    print("url could not be downloaded: " + line)
    manual.append(line)

url could not be downloaded: https://www.barrons.com/news/dr-congo-says-36-killed-in-recent-anti-un-protests-01659439507
url could not be downloaded: https://www.barrons.com/news/congo-brazzaville-president-s-party-wins-111-of-151-seats-01659480906
url could not be downloaded: https://www.barrons.com/news/11-adf-rebels-killed-in-east-dr-congo-army-01659463208
url could not be downloaded: https://www.barrons.com/news/why-are-we-being-ignored-plead-the-hungry-in-ethiopia-s-afar-01659506407
url could not be downloaded: https://www.barrons.com/news/burkina-town-blockaded-by-jihadists-warns-of-famine-01659522009
url could not be downloaded: https://www.barrons.com/news/mali-orders-foreign-forces-out-of-airport-base-after-icoast-dispute-01659463807


In [ ]:
#@title A preview of output file. Enter name for output file (.csv format) { display-mode: "form" }

file_name = 'aug03_2022.csv' #@param {type:"string"}

cols = ['url', 'title', 'summary from email', 'sample summaries','mention_of_terms']
dataa.pop(0)

df = pd.DataFrame(data=dataa, columns=cols)
df = df[df.mention_of_terms != 0]
df.head()

,url,title,summary from email,sample summaries,mention_of_terms
0,https://www.theeastafrican.co.ke/tea/oped/edit...,"Conflict, climate change, economic shocks deny...","[Opinion] Conflict, climate change, economic s...","Conflict, climate change, and economic shocks ...",2
1,https://www.dailymaverick.co.za/article/2022-0...,Africa needs to forge a unified approach to Ru...,[Opinion] Africa needs to forge a unified appr...,The trip has underscored the importance for Af...,2
4,https://apnews.com/article/africa-elections-ca...,Kenya’s election rips open scars of inequality...,Kenya’s election rips open scars of inequality...,Kenya's Aug. 9 election is ripping open the sc...,1
5,https://www.washingtonpost.com/world/2022/08/0...,A wild-card candidate in Kenya is sparking an ...,A wild-card candidate in Kenya is sparking an ...,Placeholder while article actions loadNAIROBI ...,1
13,https://www.theeastafrican.co.ke/tea/news/east...,Biden’s UN envoy in Africa tour to counter Russia,Biden’s UN envoy in Africa tour to counter Rus...,“We want African countries not to buy into Rus...,2


In [ ]:
#@title Output files is saved to "My Drive" { display-mode: "form" }

path = '/content/drive/My Drive/' + file_name

with open(path, 'w', encoding = 'utf-8-sig') as f:
  df.to_csv(f)



path = '/content/drive/My Drive/not_downloaded_' + file_name

with open(path, 'w', encoding = 'utf-8-sig') as p: 
  for line in manual:
    if 'https' in line:
      p.write(line + "\n")

FIFTH: 
- Close the folder by clicking on the small downward facing arrow to the left of the words "This file project will take in all of the emails..."
- Click the triangle "play" button to the left of the words "*9 cells hidden*"
- Once the file is running, a pop-up display will show asking if you want to connect the file to your google drive --> Sign-in to your account

# Explanation of Output file columns

***Output file only containst the urls from email that have at least ONE of the terms you mentioned above***

Columns:
- 'url', 'title', and 'summary from email': from the email
- 'sample summaries': AI generated summaries from the article's text which can aid research
- 'mention_of_terms': number of times any of the terms you mentioned above were in the article's text